In [15]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy
import numpy as np
import seaborn as sns
import xgboost
from sklearn import tree
from sklearn.tree import _tree
import pydotplus
from IPython.display import Image, display, clear_output
from ipywidgets import widgets, Layout

In [17]:
trainData = pd.read_csv("schoko_num.csv")

In [18]:
trainData = trainData / 5

In [19]:
trainData.columns= trainData.columns.str.rsplit('...').str[1] #only take the last word of each columns

In [20]:
trainData.columns= trainData.columns.str.replace('.', '') #get rid of .

In [21]:
names = trainData.columns[0:22]
sorten = trainData.columns[22:29]

In [22]:
trainX=trainData.ix[:,0:22].copy()

In [23]:
#%%time
model={}
pred={}
for sorte in sorten:
    trainY=trainData[sorte].copy()
    model[sorte] = xgboost.XGBRegressor()
    model[sorte].fit(trainX, trainY)
    #print(model[i].feature_importances_)
    pred[sorte]= model[sorte].predict(trainX)
    error=numpy.mean(numpy.abs(pred[sorte]-trainY))

In [24]:
def featureRank(sorte="Vollmilch"):
    print("Feature ranking of Sorte: " + sorte)
    importances = model[sorte].feature_importances_
    indices = np.argsort(importances)[::-1]
    namessort = names[indices]
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(trainX.shape[1]), importances[indices],
           color="purple", align="center")
    plt.xticks(range(trainX.shape[1]), namessort, rotation=90)
    plt.xlim([-1, trainX.shape[1]])
    plt.show()

In [25]:
#%%time
from sklearn import linear_model
lmodel={}
for sorte in sorten:
    trainY=trainData[sorte].copy()
    lmodel[sorte] = linear_model.RandomizedLasso(alpha=0.0004)#hier stand 0.025 oder 0.25
    lmodel[sorte].fit(trainX, trainY)

In [26]:
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)

In [27]:
clf = tree.DecisionTreeRegressor(min_samples_leaf=4) # (max_depth=4) #min_samples_leaf=4
trainYmult = trainData.ix[:,22:29]
clf = clf.fit(trainX, trainYmult)

In [28]:
from IPython.display import Image
import time

In [29]:
x = [" ",'1','2','3','4','5']
variable=0
feature_names=names
schongeklickt={}#merkt sich alle klicks, um wiederholungen zu vermeiden
    #format name: value
def redraw(turn, variable):
  
    tree_ = clf.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    def recurse(node, depth):
         
        # indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            
            if name in schongeklickt.keys():
                if float(schongeklickt[name])/5 <= threshold:                    
                    recurse(tree_.children_left[node], depth + 1)
                else:  
                    recurse(tree_.children_right[node], depth + 1)
            else:
                togl = widgets.ToggleButtons(
                    options=x,
                    description=name,
                    disabled=False,
                    button_style='', 
                    tooltip='Description')
                    #layout = Layout(width='40px')) #correction for new update - smaller width
                display(togl)
                def on_click(value):
                    if togl.value == " ":
                        pass
                    else:
                        schongeklickt[name]=togl.value
                        if float(togl.value)/5 <= threshold:
                            togl.disabled=True

                            recurse(tree_.children_left[node], depth + 1)
                        else:  
                            togl.disabled=True
                            recurse(tree_.children_right[node], depth + 1)
                togl.observe(on_click, 'value')
        else:
            ergebnisse={}
            for i in range(0, len(sorten)):
                sorte= sorten[i]
                prozent = str(tree_.value[node][i][0])
                ergebnisse[sorte]=prozent
            #    print(sorte + ": " + prozent)
            #print(sorted(ergebnisse, key=ergebnisse.get, reverse=True))
            zweisort= sorted(ergebnisse, key=ergebnisse.get, reverse = True)
            #print(max(ergebnisse, key=ergebnisse.get))
            #print(zweisort)
            schokovar = str(zweisort[0])+ ".jpg"
            display(Image(schokovar))
            schokovar2 = str(zweisort[1])+ ".jpg"
            display(Image(schokovar2))
            schokovar3 = str(zweisort[2])+ ".jpg"
            display(Image(schokovar3))
            for i in range(0,len(sorten)): 
                print(zweisort[i] + ": " + ergebnisse[zweisort[i]]) 
    recurse(0, 1)

redraw(0, 0) # initializes the first choice

A Jupyter Widget